# **PROYECTO FINAL. BIONFORMATICA**

## *PARTE 1: Descargar archivos*

Para descargar los datos de Campylobacter jejuni se puede realizar a través del link https://www.ncbi.nlm.nih.gov/sra/SRX13305038[accn]

`$ fastq-dump --split-3 SRR17120534`

Del cual se generaran dos archivos fastq (porque la libreria es pareada)


# *PARTE 2: Control de calidad*

1. Crear una carpeta con el nombre ResultsFqc y realizar el análisis de filtrado de calidad con fastqc

  `$ fastqc *.fastq -o ResultsFqc/`

2. Bajar los archivos .html a la máquima remota para visualizar de manera gráfica los resultados.

  ```
$ scp -P 10022 doliva@132.247.172.26:/home/doliva/ProyectoFinal/ResultsFqc/*.html /mnt/c/Users/Hogar/OneDrive/Documentos/fastqc
```


# *PARTE 3: Filtrado de secuencias*

Al visualizar los resultados en la interfaz de fastqc los "fallos" más notables son la gráfica de contenido de bases por secuencias (las primeras 20 bases tiene picos muy disparados) y la gráfica de distribución de longitud de la secuencia. En general si al filtrado se le aplican algunos filtrados de calidad en donde se eliminen la bases que son atípicas, estos dos grafos podrían verse en demasía mejorados.

```
$ fastp -i /home/doliva/ProyectoFinal/SRR17120534_1.fastq -I /home/doliva/ProyectoFinal/SRR17120534_2.fastq --qualified_quality_phred 30 -A -l 36 -o SRR17filtfastp_1.fastq -O SRR17filtfastp_2.fastq >& fastp_SRR17.log
```

Además de los parámetros que van por default por parte de fastp, se consideron 3 más:
  * **--qualified_quality_phred 30**. Esto quierre decir que todas aquellos reads que sobrepasen este número serán eliminados

  * **-A**. Que elimina todos los adapatadores de manera automática y sin restricción

  * **-l 36**. Length_required, se descartarán todos las bases que estén por arriba de 36

Una vez que los nuevos datos .fastq han sido generados, se tiene que realizar otro análisis de calidad con fasqc para verificar que los parámetros sean más aceptables.

# *PARTE 4: Cálculo de cobertura*

Al escuchar la palabra cobertura, se enfatiza en la profundidad del genoma. En promedio, cuantas veces se está leyendo cada una de las bases que cubren el genoma.

 ```
  C=LN/G 
    * C: coverage
    * L: read lenght
    * N: Number of reads
    * G: Genome lenght
    * LN: cantidad total de pb en los reads
```

1. A partir de los archivos creados con el filtrado de fastp, se calcula el # de reads.
Observar la cantidad de líneas de los archivos FileOfiltfastp_1.fastq FileOfiltfastp_2.fastq
el # de líneas divididas entre 4 nos dara el # total de reads.

  ```
$ wc -l /home/doliva/ProyectoFinal/Fastp/*.fastq
   523720 /home/doliva/ProyectoFinal/Fastp/SRR17filtfastp_1.fastq
   523720 /home/doliva/ProyectoFinal/Fastp/SRR17filtfastp_2.fastq
  1047440 total
```

  ```
$ expr 1047440 / 4
261860
```

2. Multiplicar ese 261860 por el largo de la secuencia (En Illumina casi siempre todas las secuencias miden lo mismo)

  ```
$ expr 21860 * 251
5486860
```		
  
Así pues 5486860 es la cantidad total de pb que hay en los reads


3. Para calcular la longitud del genoma de referencia utilizaremos scriptome

  ```
$ perl -e ' $count=0; $len=0; while(<>) { s/\r?\n//; if (/^>/) { $count++; } else { $len += length($_) } } print "Read $count FASTA records in $. lines. Total sequence length: $len\n"; warn "\nRead $count FASTA records in $. lines. Total sequence length: $len\n\n"; ' CampylobacterColi.fa > Campylobactercount.txt
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
        LANGUAGE = (unset),
        LC_ALL = (unset),
        LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").

  Read 1 FASTA records in 23980 lines. Total sequence length: 1678432
```

4. Luego aplicando la fórmula

  ```
C=LN/G
C=5486860/1678432
C=3.269x
```

# *PARTE 5: Mapeo*

1.  Hacer el índice del genoma de referencia (CampylobacterColi.fa) con bowtie2.
  
  Los índices son muy útiles para explorar de manera más fácil dentro de la base de datos.

  `conda activate bowtie2`

  `$ bowtie2-build -f /home/doliva/ProyectoFinal/Mapeo/CampylobacterColi.fa CampylobacterColi`


2.  Correr el mapeo.

  ```
$ bowtie2 --maxins 1000 -x CampylobacterColi -1 /home/doliva/ProyectoFinal/Fastp/SRR17filtfastp_1.fastq -2 /home/doliva/ProyectoFinal/Fastp/SRR17filtfastp_2.fastq -S CampColi.sam
```
  *  S para generar un archivo .sam
  *  -x/ --maxins es el nombre del índice de referencia que hemos creado en el paso anterior.
  *  Este paso en partícular es un poco tardado debido a que el programa lo que hace es buscar en toda la secuenica de referencia los reads pareados que coinciden con ella, no es sencillo si la secuencia de interés contiene miles de pares de bases. 


3.  Filtrar el archivo .sam que se generó. Dicho filtrado eliminará aquellos reads que no lograron mapearse, es decir no encontraron una secuencia específica del genoma de referencia con la cual alinearse.

  `awk '$3!="*"' CampColi.sam >CampColiFilt.sam`

# *PARTE 6: Ensamble de genoma*

Se utilizará el programa SPAdes que pertenece al algoritmo DeBruijn Graph. 

Su forma de trabajar es partiendo los reads en pezados más cortos a los que denomina k-meros, posteriormente se analiza que k-meros van traslapando para ir generando el ensamble. Los k-meros tienen que ser los suficientemente largos para tener un ensamble más grande y confiable y además tienen que ser números nones, de otra forma, si son pares generarían regiones palíndrómicas que no brindarán información relevante en la anotación.

1.  Ensamble de archivos fastq con

  ```
  $ conda activate spades

  $ spades.py -k 33,37,41 -t 1 -m 7 --pe1-1 SRR17filtfastp_1.fastq --pe1-2 SRR17filtfastp_2.fastq -o spades_SRR

  $ conda deactivate
```

Se genera una carpeta  spades_SRR que contiene varios archivos, entre ellos los scaffolds

2. Checar la calidad del ensamble con quast

  ```
  $ quast --split-scaffolds -t 1 scaffolds.fasta
```

Genera automaticamente un directorio llamado quast_results en cual contiene los resultados en html.

3. Descargar los resultados del análisis de calidad con quast

  ```
  $ scp -P 10022 -r doliva@132.247.172.26:/home/doliva/ProyectoFinal/GenomeAssembly/spades_SRR/quast_results/results_2021_12_06_15_42_56 .
```

En la máquina remota abrir los archivos .html que contienen algunos gráficos y estadísticos de los scaffolds y contigs.

# *PARTE 7: Anotación (de una porción)*

A partir de aquí trabajará con el archivo scaffold.fasta que contiene los múltiples scaffolds que generó el ensamblado.

1. Se verifica el número de scaffolds  

  ```
$ grep '>' -c scaffolds.fasta
38
```

2. La porción que se va a anotar será el scaffold #3

  * Primero se transforma el archivo .fasta a formato tabular .tab

    ```
$ perl -e ' $count=0; $len=0; while(<>) { s/\r?\n//; s/\t/ /g; if (s/^>//) { if ($. != 1) { print "\n" } s/ |$/\t/; $count++; $_ .= "\t"; } else { s/ //g; $len += length($_) } print $_; } print "\n"; warn "\nConverted $count FASTA records in $. lines to tabular format\nTotal sequence length: $len\n\n"; ' scaffolds.fasta > scaffolds.tab
```

  * Se cortó y guardó en un nuevo archivo.tab unicamente las secuencias de los scaffolds

    ```
$ perl -e ' $col=-1; while (<>) { s/\r?\n//; @F = split /\t/, $_; $len = length($F[$col]); print "$_\t$len\n" } warn "\nAdded column with length of column $col for $. lines.\n\n"; ' scaffolds.tab > scaffolds_length.tab
```

  * Se creó otro archivo.cat que contiene únicamente el ID del scaffold 3

    ```
$ cat > Scaffold03.list
NODE_3_length_197711_cov_33.605211
```

  * Con el siguiente comando se eligió la secuencia del scaffold 3 del archivo .fasta orginal y se guardó en un nuevo archivo .fna

    ```
$ perl -e ' ($id,$fasta)=@ARGV; open(ID,$id); while (<ID>) { s/\r?\n//; /^>?(\S+)/; $ids{$1}++; } $num_ids = keys %ids; open(F, $fasta); $s_read = $s_wrote = $print_it = 0; while (<F>) { if (/^>(\S+)/) { $s_read++; if ($ids{$1}) { $s_wrote++; $print_it = 1; delete $ids{$1} } else { $print_it = 0 } }; if ($print_it) { print $_ } }; END { warn "Searched $s_read FASTA records.\nFound $s_wrote IDs out of $num_ids in the ID list.\n" } ' Scaffold03.list scaffolds.fasta > Scaffold03.fna
```


**Anotación estructural**

La anotación estructural se realizó utilizando Augustus, un programa que predice genes en secuencias genómicas

Para los parámetros, se modifico el organismo más cercano eligiendo  E.Coli, se especificó que elreporte de los genes fuera en ambas hebras, además de un porcentaje medio de splicin alternativo.

En la terminal, crear dos archivos .fasta con el reporte de las secuencias codificantes y las de los aminoacidos

`$ cat > aaseqScaffold3.fa`

`$ cat > seqcodScaffold3.fa`

Se predijeron 82 genes 

```
$ grep '>' -c seqcodScaffold3.fa
82
```

**Anotación funcional**

Se realizaron dos blast, el primero con 20 proteínas de Campylobacter Coli y el segundo con varias proteínas del genoma de un gallo.

`$ makeblastdb -in ref_seqGallus.fa -dbtype prot out GallusDB`

`$ blastp -db GallusDB -query aaseqScaffold3.fa -out ScaffoldVsGallusDB`

`$ makeblastdb -in ref_seqCampColi.fa -dbtype prot -out CampylobacterColiDB`

```
$ blastp -db CampylobacterColiDB -query aaseqScaffold3.fa -out ScaffoldVsCampColiDB
```

Evidenemente hubo más alineamientos con la base de datos de Campylobacter Coli, pero me llamó la atención que de tantas secuencias de proteínas del organismo Gallus hubo algunas proteínas helicasas con las que si hubo alineamiento.
